# Creating a data base table from a file containing neuron ids

This notebook contains code to generate a data base table from scratch, using only a list of IDs from a txt file and the available information stored in the FlyWire server

In [3]:
import os
import pandas as pd
import numpy as np
from fafbseg import flywire
from caveclient import CAVEclient
client = CAVEclient('flywire_fafb_production')

## A) Choosing file, loading ids and creating data base table


### Chossing the txt file of interest and id update

In [13]:
#Loading ids
dataPath = r'E:\Connectomics-Data\FlyWire\Txts'
fileName = 'root_ids_Tm9_class_optic_lobes_side_right.txt'
filePath = os.path.join(dataPath,fileName)
ids_df = pd.read_csv(filePath, sep = ",")
initial_ids_list = ids_df.columns.tolist()

In [22]:
#Updating ids
ids_update_df = flywire.update_ids(initial_ids_list, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)

Updating:   0%|          | 0/698 [00:00<?, ?it/s]

In [61]:
#Filtering out bad updates
filtered_id_update_df = ids_update_df[ids_update_df['confidence'] >= 0.95]
ids_list = filtered_id_update_df['old_id'].tolist()
updated_ids_list = filtered_id_update_df['new_id'].tolist()

In [77]:
# Getting id identification
identification_df = flywire.find_celltypes(updated_ids_list, user=None, exact=False, case=False, regex=True, update_roots=False)
identification_df.head()

,id,created,superceded_id,pt_position_x,pt_position_y,pt_position_z,tag,user_id,pt_supervoxel_id,pt_root_id
46445,69136,2022-12-14 12:27:39.139449+00:00,NaN,707164,316212,83400,"Tm9, Tm9_L, transmedullary neuron Tm9, Choline...",153,82758522504262702,720575940626174522
46450,69138,2022-12-14 12:28:08.861083+00:00,NaN,703616,298688,79480,"Tm9, Tm9_L, transmedullary neuron Tm9, Choline...",153,82687878815586003,720575940630668871
55703,66360,2022-11-28 17:15:24.221904+00:00,NaN,764796,360224,106880,"Tm9, Tm9_L, transmedullary neuron Tm9, FBbt_00...",2395,83744440904245193,720575940627187849
55711,66362,2022-11-28 17:26:36.339709+00:00,NaN,776116,176348,133520,"Tm9, Tm9_L, transmedullary neuron Tm9, FBbt_00...",2395,83952454827425104,720575940633362131
57859,66361,2022-11-28 17:24:46.608047+00:00,NaN,783032,176784,141840,"Tm9, Tm9_L, transmedullary neuron Tm9, FBbt_00...",2395,84093192315805909,720575940612657522


## Creating the data base table

In [78]:
# Initializing the data set from a dictionary
initial_dict = {}
initial_dict['seg_id'] = identification_df['pt_root_id'].astype(str).tolist()
initial_dict['author'] = identification_df['user_id'].astype(str).tolist()
initial_dict['name'] = identification_df['tag'].astype(str).tolist()
df = pd.DataFrame(initial_dict)



## Saving table in an excel file

In [82]:
#Saving in a new file

df = df.astype(str)
import datetime
x = datetime.datetime.now()
date_str = x.strftime("%d") + x.strftime("%b") + x.strftime("%Y")
file_name = f'new_table_{fileName}_{date_str}.xlsx'
savePath = os.path.join(dataPath, file_name)
df.to_excel(savePath, sheet_name='Initial table')